In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *

In [ ]:
# context = Context('amended-1-no_noise', debug=0, dep_var=dep_var2, mulr=0, load_data=False)
# context = Context('amended-3-less-features-no-anomaly-7', debug=0, dep_var=dep_var2, mulr=7, load_data=False)
context = Context('amended-3-less-features-no-anomaly-7', debug=0, dep_var=dep_var2, mulr=7, load_data=False)

# data

In [ ]:
# list existing data files
print('Existing data files:', [o.stem for o in Path(context.fn_feather).parent.glob('*.feather')])

fmtr = DataFormatter(context, cycle_feathers=context.fn_cycles)
cycles = fmtr.cycles # shorten notation
# fmtr.drop_redundent_columns() # already done for cycles
idx = fmtr.beginning_index()

In [ ]:
# anomaly range
a_range = pd.read_csv('tbmData/不良地质条件桩号范围.csv', header=None)

In [ ]:
a_range.iloc[:,0] - a_range.iloc[:,1]

In [ ]:
less_features

In [ ]:
less_features = dep_var1 + dep_var2 + ['桩号', '给定频率', '刀盘速度给定']
less_features = tuple(less_features)
cycles_less = cycles.loc[:,less_features]
cycles_less.head()

In [ ]:
t_range = [(cycles_less.loc[i,'桩号'].iloc[0], cycles_less.loc[i,'桩号'].iloc[-1]) for i in cycles_less.index.levels[0]]
t_range = pd.DataFrame(t_range)

In [ ]:
a_idx = []
for a_r in a_range.values:
    is_in = (a_r - t_range).product(1) < 0
    a_idx.extend(np.where(is_in.values)[0].tolist())

# a_idx = tuple(a_idx)

In [ ]:
len(a_idx)

In [ ]:
t = cycles_less.drop(index=list(a_idx))
# t = t.reset_index(level=0, drop=True)

cycles_less = t

In [ ]:
plots(cycles_less.loc[735]);

target data format: `[flattened matrix: 30x150, extra cont: 1x2, extra cat: n]`

parameters: cat_names

Use no cat for now

In [ ]:
fmtr.cycles = cycles_less

In [ ]:
df_conts, stat_x = fmtr.get_x(normalize=True)
extra_x = fmtr.get_extra_x_task2()
deps = fmtr.get_y()
stat_y = [o.loc[context.dep_var] for o in stat_x]
stat_extra_x = [o.loc[dep_var1] for o in stat_x]
train_cont = flatten_and_cat(df_conts, [extra_x, deps], sl=context.sl)

In [ ]:
train_cont.to_feather(context.fn_feather)
np.savez_compressed(context.fn_np, idx=idx, stat_x=stat_x, stat_y=stat_y, stat_extra_x=stat_extra_x)
context.fn_feather

In [ ]:
# context = Context('amended-1-no_noise', debug=0, dep_var=dep_var2, mulr=1, gpu_start=9)
# context = Context('amended-2-7', debug=0, dep_var=dep_var2, mulr=7, gpu_start=8)
context = Context('amended-3-less-features-7', debug=0, dep_var=dep_var2, mulr=7, gpu_start=8)
databunch = MultiDeptTabularDataBunch.from_df('tmp', context.cyc_cont, context.dep_var, valid_idx=context.valid_idx_tile, bs=context.bs)

In [ ]:
x, y = ni(databunch.train_dl)
[o.shape for o in x], y.shape

In [ ]:
x[1]

# model

In [ ]:
hyper_params = {
    'use_extra_x': 1,
    'n_hidden': 400,
    'n_layers': 3,
    'hidden_p': 0.3,
    'input_p': 0.4,
    'weight_p': 0.5,
    'drops': [0.4] * 4,
    'alpha': 2.,
    'beta': 1.,
    'clip': 0.5,
    'loss_func': MSELossFlat()
}
hyper_params['layers'] = [3*(context.n_cont) + int(hyper_params['use_extra_x']) * 3, 5*context.n_cont, 5, len(context.dep_var)]

In [ ]:
emb_sz = 3 # use rule of thumb later
model = Task2Model(context.n_cont,emb_sz, hyper_params['n_hidden'], hyper_params['n_layers'], hyper_params['layers'], hyper_params['drops'], 
                  hidden_p=hyper_params['hidden_p'], input_p=hyper_params['input_p'], weight_p=hyper_params['weight_p'], use_extra_x=hyper_params['use_extra_x'])
model = model.cuda()

p, raw_outputs, outputs = model(*x)

In [ ]:
learner = get_new_model(context, databunch, hyper_params)

In [ ]:
learner.lr_find()

learner.recorder.plot()

In [ ]:
learner.fit(30, lr=1e-1)

In [ ]:
learner.fit(40, lr=1e-2)

In [ ]:
learner.fit(20, lr=1e-2)

In [ ]:
learner.save('new_model_test')

In [ ]:
npz = np.load(context.fn_np)

In [ ]:
npz['stat_y']

In [ ]:
extra = denormalize(context.cyc_cont.iloc[:,-4:-2], *npz['stat_extra_x'])

In [ ]:
extra.mean(), extra.std()